In [4]:
import os
import pandas as pd
import datetime

current_date = pd.Timestamp(datetime.date.today())

def calculate_rolling_average(df, columns, window=10):
    # Calculate the rolling mean for the specified columns
    rolling_df = df[columns].rolling(window=window, min_periods=10).mean()
    #hift the rolling mean by 1 to not include the current game's stats in the average
    return rolling_df

# Base path for the original data
base_path = '23_24_Team'
df4 = pd.DataFrame()
# New top-level folder for averaged data
average_data_path = '23_24_test'
os.makedirs(average_data_path, exist_ok=True)
# Iterate over each year's folder
# Iterate over each team's CSV file
for team_file in os.listdir(base_path):
    if team_file.endswith('.csv'):
        team_path = os.path.join(base_path, team_file)
        # Read the CSV file
        df = pd.read_csv(team_path)

        # Select the desired columns and calculate the rolling averages
        selected_columns = ['Game', 'Team', 'Date', 'AwayTeam', 'AwayScore', 'HomeTeam', 'HomeScore', 'HomeResult', 'days_of_rest']
        other_columns = df.columns.difference(selected_columns)
        rolling_df = calculate_rolling_average(df, other_columns)

    
        # Combine the selected columns with the rolling averages
        combined_df = pd.concat([df[selected_columns], rolling_df], axis=1)
        cols_to_drop = ['Game','AwayTeam', 'AwayScore', 'HomeTeam', 'HomeScore', 'HomeResult','days_of_rest']
       
    #drop other unneccesary columns
        combined_df.drop(columns=cols_to_drop, inplace=True)
        combined_df['Date'] = pd.to_datetime(combined_df['Date'])
        combined_df['time_diff']=(current_date-combined_df['Date'])
        combined_df['days_of_rest'] = combined_df['time_diff'].dt.days - 1
        combined_df.drop('time_diff', axis=1, inplace=True)
        days_of_rest_column = combined_df.pop('days_of_rest')

        combined_df.insert(2, 'days_of_rest', days_of_rest_column)
        last_row_df = combined_df.tail(1)
        df4 = pd.concat([df4, last_row_df], ignore_index=True, axis=0)
        # Save the new DataFrame as a CSV file in the year-specific folder under 'average_data'

#df4.head()
df4.to_csv('test.csv', index=False)

In [3]:
df4['Date'] = pd.to_datetime(df4['Date'])

        # Calculate the difference between each game's date
df4['time_diff'] = df4['Date'].diff()

# Convert the timedelta object to integer for days of rest
df['days_of_rest'] = df['time_diff'].dt.days - 1
df.drop('time_diff', axis=1, inplace=True)
days_of_rest_column = df.pop('days_of_rest')

# Reinsert the 'days_of_rest' column into the third position (position 2)
df.insert(2, 'days_of_rest', days_of_rest_column)


KeyError: 'time_diff'